In [1]:
import os, sys, warnings
import dill
import sqlalchemy
import traceback
import ipywidgets as widgets
from IPython.display import HTML
import base64
from aepsych.plotting import plot_strat
from aepsych.server import AEPsychServer
from aepsych.server.sockets import PySocket
from IPython.display import display,  FileLink
from ipywidgets import HTML, AppLayout, Box, Button, Label, Layout, Text
from IPython.core.display import HTML as html
import pandas as pd
import numpy as np
from tempfile import NamedTemporaryFile, TemporaryDirectory
from zipfile import ZipFile
import matplotlib.pyplot as plt

test = widgets.Output(layout=Layout())
display(test)


# plt.rcParams["figure.figsize"] = (10, 10)
plt.rcParams["figure.facecolor"] = "white"
warnings.filterwarnings("ignore")

connect_out = widgets.Output(layout=Layout(padding="2px", height="150px"))
!true > logs/bayes_opt_server.log #  clears the server.log file data when you start
# !rm /var/svcscm/databases/*.* # deletes all the db files

with connect_out:
    server = None
    connect_out.clear_output()
strat = None
db_file_name = None
db_config_str = None
database_path = None
new_experiment = False
resume_experiment = False
dim = 0



#---------- Style ----------
style = {"description_width": "initial"}

display(html("""<style>
.jp-CodeCell {
    width:100% !important;
    background: whitesmoke;
}
.jp-Notebook .jp-Cell {
    padding: 0 !important;
}

.jupyter-widgets.widget-tab {
    margin: 40px 30px;
}

.p-Collapse-header,
.jupyter-widgets.widget-tab > .p-TabBar .p-TabBar-tab.p-mod-current:before,
.jupyter-button ,
.widget-button,
.jupyter-widgets.widget-tab > .widget-tab-contents {
    border-radius: 4px;
    border: 1px solid #dcdcdc;
}


.jupyter-widgets.widget-tab > .p-TabBar .p-TabBar-tab{
border-top-right-radius:4px;
border-top-left-radius: 4px;
border: 1px solid #dcdcdc;
}
.p-Collapse-contents{
border-bottom-right-radius:4px;
border-bottom-left-radius: 4px;
border: 1px solid #dcdcdc;
border-top:none;
}

.p-Collapse-open > .p-Collapse-header {
border-bottom-right-radius:0px;
border-bottom-left-radius: 0px;
}
body.jp-Notebook {
    margin: 0 !important;
    padding: 0 !important;
    height: 100vh !important;
    background: whitesmoke;
}
.widget-inline-hbox {
    margin: 0px;
}
.widget-output {
   # padding: 10px !important;
   # background: white;
   # border-radius: 4px !important;
   # box-shadow: inset 0 1px 1px rgb(0 0 0 / 5%);
   # border: 1px solid #e3e3e3;
   # border-radius: 4px;
   # -webkit-box-shadow: inset 0 1px 1px rgb(0 0 0 / 5%);
   # box-shadow: inset 0 1px 1px rgb(0 0 0 / 5%);
   # margin: 10px;
   # max-width: 600px; 
    height: auto !important;
    width: auto !important;
   # overflow-y: auto;
}

.jp-OutputArea {
    overflow-y: inherit;
}

.widget-text input[type="text"],
.widget-text input[type="number"],
.widget-dropdown > select {
   border-radius: 4px;
   box-shadow: inset 0 1px 1px rgb(0 0 0 / 5%);
}

.widget-vbox {
    justify-content: center;
    width: 100%;
    width: -webkit-fill-available !important;
    display: flex;
    border-radius: 4px;
    background: white;
    height: fit-content;
    padding: 20px;
   # box-shadow: 0px 1px 3px rgb(0 0 0 / 12%), 0px 1px 2px rgb(0 0 0 / 24%);
}


[data-jp-theme-light='true'] .jp-RenderedImage img.jp-needs-light-background {
    width: 100%;
}


.jupyter-widgets.widget-tab {
    min-width: 30% !important;
}
html {
    background: #F2F2F2;
    height: 100%;
    margin: 0px;
    padding: 0px;

}

.widget-dropdown,
.widget-text{
    width: auto;

}
.widget-label {
    display: flex;
    align-items: center;
}
.jp-RenderedText[data-mime-type='application/vnd.jupyter.stderr'] {
    background: white;
    padding-top: var(--jp-code-padding);
}
.jupyter-widgets.widget-tab > .widget-tab-contents {
    border-radius: 4px;
    border-top-left-radius: 0px;
    border-top-right-radius: 0px;
}

.widget-box {
    overflow: hidden !important;
}
.jp-OutputArea-output {
    overflow: hidden
    }
.jp-RenderedHTMLCommon p {
    margin-bottom: 0;
}
div.output_stderr {
    background: #fdd;
    display: none;
}

.main_container,
.connect_container_color {
background-color:  #F5F5F5;
margin-top: 3px;
padding:0; 
}

.plot_inputs {
width: 200px;
}

.init_strat_label,
.opt_strat_label,
.param_label{
font-weight: 800;
}

.config_label {
font-weight: 900;
font-size: 15px; 
display:flex; 
justify-content: center; 
}
.jp-RenderedImage img {
    max-width: 100%;
    height: auto;
    width: 750px;
    margin-righ: 20px; 
}
.plot-container {
    min-width: 750px;
}
.download-link{
    color: #192bc3; 
}
.download-link:hover{
    color: blue; 
}
</style>"""))


header = HTML(
    """
<div class="nav" style=" 
margin: 0px;
background-color: #F2F2F2;
font-weight: bold; 
box-shadow: 0px 1px 3px rgb(0 0 0 / 12%), 0px 1px 2px rgb(0 0 0 / 24%);
">
    <h1 style="
    text-align: center;
    font-family: helvetica;
    font-weight: 400;
    font-size: 20px;
    display: flex;
    align-items: center; 
    padding: 15px 20px;
    margin: 0px;
    hieght: 100px; 
    jusify-content: flex-start;
    color:rgb(80,103,132)">
    <img src="./logo.png" alt="logo" width="50" height="50" style="margin-right: 10px;" >
    AEPsych Visualizer
    </h1>
</div>
"""
)

logs_style = Layout(width='600px', height='400px')
input_style = {"description_width": "initial"}
input_layout= Layout(margin="5px 0")
btn_style = Layout(margin="10px",width="95%")

btn_box_layout = Layout(display="flex", justify_content="flex-end")
file_output = widgets.Output(
    layout={
            "overflow": "scroll", 
            "height": "50vh",
            "width": "100%",
            "padding": "5px"
           }
)

# ---------- Inputs -----------

def on_value_change(change):
    file_output.clear_output()
    with file_output:
        change["new"]

        
def on_value_change_input(change):
    inputs_output.clear_output()
    with inputs_output:
        change["new"]   


yes_label = Text(
    value="Yes Trial", placeholder="yes_label",
    description="yes_label: ", disabled=False, style=input_style,
    layout=input_layout
)

no_label = Text(
    value="No Trial", placeholder="no_label", 
    description="no_label: ", disabled=False, style=input_style,
    layout=input_layout
)

target_level = widgets.BoundedFloatText(
    value=0.75, min=0, max=1, 
    step=0.1, description="target_level:", disabled=False, 
    style=input_style, layout=input_layout
)

cred_level = widgets.BoundedFloatText(
    value=0.95, min=0, max=1, step=0.1, 
    description="cred_level:", disabled=False, 
    style=input_style, layout=input_layout
)

xlabel = Text(
    value="Angle (degrees)", placeholder="x axis label",
    description="xlabel: ", disabled=False, style=input_style,
    layout=input_layout
)

ylabel = Text(
    value="Detection Probability", placeholder="y axis label", 
    description="ylabel: ", disabled=False, style=input_style,
    layout=input_layout
)

# Observes input changes

inputs_output = widgets.Output(
    layout={"border": "1px solid black", "padding": "20px", "margin": "20px", "min-height ": "100px"}
)
yes_label.observe(on_value_change_input, names="value")
no_label.observe(on_value_change_input, names="value")
target_level.observe(on_value_change_input, names="value")
cred_level.observe(on_value_change_input, names="value")
xlabel.observe(on_value_change_input, names="value")
ylabel.observe(on_value_change_input, names="value")


# -------- ports ----------
server_ip = widgets.Text(
    placeholder="0.0.0.0", # must use regex to check pattern and type int
    description='Server IP',
    disabled=False,
    layout=Layout(display='flex', flex_flow="column", margin="10px"))
port = widgets.Text(
    placeholder="5555", # must add regex, type check, and error handling
    description='Port',
    disabled=False,
    layout=Layout(display='flex', flex_flow="column", margin="10px", justify_content="center"))

connected_ip = widgets.Output(ayout=Layout(display="flex", justify_content="flex-end"))

def on_change_server_input(change):
    with inputs_output:
        change["new"]

                              
server_ip.observe(on_change_server_input, names="value")
port.observe(on_change_server_input, names="value") 

button_submit_port_ip = widgets.Button(
    description="Start Server",
    disabled=False,
    button_style="info",
    tooltip="submit",
    layout=Layout(align_self="flex-end", width="fit-content", margin="10px"),
)


def mv_db_file(server_ip):
    mv_path = f"!mv {server_ip} /Users/ecortez/Work/voila/notebooks/databases" 
    return mv_path


def submit_port_ip_clicked(b):
    # This is for new experiments
    global server
    connected_ip.clear_output()
    try:
        if len(port.value) and len(server_ip.value):
            server = AEPsychServer(port.value, server_ip.value)
            print(mv_db_file(server_ip.value))
            
            with connected_ip:
                print(f"Connected by Client at {server_ip.value}")
        else:
            print("Server is connected to default port and IP address. Enter a port or ip address.")
    except Exception as e:
        print(f"Unable to connect at ip: {server_ip.value} and port {port.value}")
        print(e)
        print(traceback.format_exc())


button_submit_port_ip.on_click(submit_port_ip_clicked)
connection_input_box = widgets.Box(
    [server_ip, port, button_submit_port_ip], layout=Layout(justify_content="flex-end", margin="4px")
)
connection_inputs = widgets.VBox([connection_input_box, connected_ip], 
                                 layout=Layout(min_height="15px", display="flex", justify_content="flex-end"))

connection_inputs.add_class("connect_container_color")
# -------- Config --------
    

config_output = widgets.Output()

button_submit_config = widgets.Button(
    description="Submit Config File",
    disabled=False,
    button_style="info",
    tooltip="submit",
)

# CONFIG INPUTS
def add_param(b):
    global dim
    dim += 1
    hb = widgets.HBox(
        [
            widgets.Text(f"par{dim}", description="Name", layout=Layout(width="fit-content"), style=style),
            widgets.FloatText(
                0.0, description=" Lower Bound:", layout=Layout(width="fit-content"), style=style),
            widgets.FloatText(
                1.0, description=" Upper Bound:", layout=Layout(width="fit-content"), style=style),
        ],
        layout=Layout(margin="2px 0",display='flex',justify_content='flex-start',width="fit-content")
    )
    params_boxes.children = tuple(list(params_boxes.children) + [hb])
    pars = [child.children[0].value for child in params_boxes.children]
    lbs = [child.children[1].value for child in params_boxes.children]
    ubs = [child.children[2].value for child in params_boxes.children]

        
def remove_param(b):
    global dim
    if dim > 1:
        dim -= 1
        params_boxes.children = tuple(list(params_boxes.children[:-1]))

acq_dict = {
    "Exploration": "MonotonicMCPosteriorVariance",
    "Optimization": "qNoisyExpectedImprovement",
    "Threshold Finding": "MonotonicMCLSE",
}

config_label = widgets.Label(value="Configure Server")
config_label.add_class("config_label")

strategy_btns = widgets.RadioButtons(
    options=["Threshold Finding", "Exploration", "Optimization"],
    value="Threshold Finding",
    description="Strategy:",
)
        
params_label = widgets.Label(value="Parameters:")
params_label.add_class("param_label")
params_boxes = widgets.VBox([], layout = Layout(padding="2px"))
add_param(None)


add_param_btn = widgets.Button(description="Add Parameter")
add_param_btn.on_click(add_param)

rem_param_btn = widgets.Button(description="Remove Parameter")
rem_param_btn.on_click(remove_param)

params_btns = widgets.HBox([add_param_btn, rem_param_btn], layout = Layout(margin="24px 0 4px 0"))

threshold_input = widgets.BoundedFloatText(
    value=0.75, min=0, max=1.0, step=0.05, description="Threshold:",
    style=style, layout=Layout(width="fit-content", margin="4px 0")
)

stimuli_per_trial = widgets.Dropdown(
    options=[('1 for single', 1), ('2 for pairwise experiments', 2)],
    value=1,
    description='Stimuli per trial: ',style=style,
    layout=Layout(width="fit-content", margin="4px 0")
)

outcome_types = widgets.Dropdown(
    options=[('binary', "[binary]"), ('continuous', "[continuous]")],
    value="[binary]",
    description='Outcome types: ', style=style, 
    layout=Layout(width="fit-content", margin="4px 0")
)
init_strat_label = Label("Init Strategy")
init_strat_label.add_class("init_strat_label")
init_min_total_tells = widgets.IntText(value=5, description="Min total tells: ",style=style,
                                              disabled=False, layout=Layout(width="fit-content", margin="4px 0"))

opt_strat_label = Label("Opt Strategy")
opt_strat_label.add_class("init_strat_label")
opt_min_total_tells = widgets.IntText(value=50, description="Min total tells: ",style=style,
                                              disabled=False, layout=Layout(width="fit-content", margin="4px 0"))

refit_every = widgets.IntText(value=5, description="Refit every: ",style=style,
                                              disabled=False, layout=Layout(width="fit-content", margin="4px 0"))


config_container = widgets.VBox([config_label,strategy_btns, threshold_input, 
                                 stimuli_per_trial,outcome_types,
                                 init_strat_label,init_min_total_tells,
                                 opt_strat_label, opt_min_total_tells,refit_every,
                                 params_btns,
                                 params_label, params_boxes], 
                                layout=Layout(padding="8px", margin="0 2px 10px 2px"), 
                               )



def make_config():
    global server
    global dim 
    
    dim = len(params_boxes.children)
    pars = [child.children[0].value for child in params_boxes.children]
    parnames = f"[{','.join(par for par in pars)}]"
    lbs = [child.children[1].value for child in params_boxes.children]
    ubs = [child.children[2].value for child in params_boxes.children]
    
    target = threshold_input.value
    acq = acq_dict[strategy_btns.value]
    model = "GPClassificationModel" if acq == "qNoisyExpectedImprovement" else "MonotonicRejectionGP"
    generator = "OptimizeAcqfGenerator" if acq == "qNoisyExpectedImprovement" else "MonotonicRejectionGenerator"
    
#     other inputs 
    stimuli = stimuli_per_trial.value
    outcome_type = outcome_types.value
    init_min_tells = init_min_total_tells.value
    opt_min_tells = opt_min_total_tells.value
    refit = refit_every.value
    config_str = f"""
    [common]
    parnames = {parnames} # names of the parameters
    lb = {lbs} # lower bound of the parameter
    ub = {ubs} # upper bound of parameter
    stimuli_per_trial = {stimuli} # the number of stimuli shown in each trial; 1 for single, or 2 for pairwise experiments
    outcome_types = {outcome_type} # the type of response given by the participant; can be [binary] or [continuous]
    target = {target} # desired threshold, for threshold estimation.
    strategy_names = [init_strat, opt_strat] # The strategies that will be used, corresponding to the named sections below

    # Configuration for the initialization strategy, which we use to gather initial points
    # before we start doing model-based acquisition
    [init_strat]
    min_total_tells = {init_min_tells} # number of sobol trials to run
    generator = SobolGenerator # The generator class used to generate new parameter values

    # Configuration for the optimization strategy, our model-based acquisition
    [opt_strat]
    min_total_tells = {opt_min_tells} # total number of trials to run
    refit_every = {refit} # how often to refit the model from scratch
    generator = {generator} # The generator class used to generate new parameter values
    acqf = {acq} # The acquisition function; MCLevelSetEstimation is used for threshold finding
    model = {model} # The model class"""
    
    config = config_str
    
    try:
        config_output.clear_output()
        with config_output:
            if server is not None:
                server.configure(config_str=config)
                print("Config sent to server successfuly")
            else:
                print("Server not connected")
    except Exception as e:
        print("Unable to configure server. Please reset and reconfigure.")
        print(e)
        print(traceback.format_exc())


def clicked_submit_config(b):
    make_config()


button_submit_config.on_click(clicked_submit_config)

config_buttons = widgets.HBox(
    [button_submit_config],
    layout=Layout(display="flex", justify_content="flex-end", marging="5px 0"),
)


# Resume Config
resume_config_output = widgets.Output(layout=Layout(margin="10px"))

def resume_config_value():
    global db_config_str
    resume_config_output.clear_output()
    with resume_config_output:
        display(html(f"""
        <div style='height: 400px;'>
            <pre
            style='font-family: monospace;background-color: black;
            color: green;padding: 8px;height: 100%;overflow: auto; '
            >{db_config_str}</pre>

        </div>
        """))


# -------- Plot view ----------

# --------- Accordion -----------
button_submit = widgets.Button(
    description="Submit", disabled=False, 
    button_style="info", tooltip="submit", 
    layout=btn_style
)
accordion = widgets.Accordion(
    children=[
        widgets.VBox([no_label, yes_label], layout=Layout(padding="5px")),
        widgets.VBox(
            [
                target_level,
                cred_level,
            ],
            layout=Layout(padding="5px")
        ),
        widgets.VBox([xlabel, ylabel],layout=Layout(padding="5px")),
    ],
    layout=Layout(min_width="280px")
)
accordion.set_title(0, "Outcome Labels")
accordion.set_title(1, "Parameters")
accordion.set_title(2, "Axis Labels")





def submit():
    file_output.clear_output()
    with file_output:
        display_plot()
        
        
def submit_clicked(b):
    submit()

    
button_submit.on_click(submit_clicked)

plot_interact_box = widgets.VBox([accordion, button_submit])
plot_interact_box.add_class("plot_inputs")
# -------- Logging --------
import logging
import aepsych.utils_logging as utils_logging

logger_output = widgets.Output()

log_path = os.getcwd() + "/logs/bayes_opt_server.log"


def load_logs():
    logger_output.clear_output()
    log_str = ""
    logger_output.clear_output()
    with logger_output:
        with open(log_path) as f:
            contents = f.readlines()
            for log in contents:
                log_str += f"<p style='margin-top: 1px;margin-bottem: 1px'>{log}</p>"
        display(html(f"""
        <div style='height: 400px;overflow: auto;'>
            <pre
            style='font-family: monospace; background-color: black; color: white; 
            padding: 4px; height: 100%;'
            >{log_str}</pre>
        </div>
        """))
                

# --------- Interactive ------------

param_output = widgets.Output(
    layout=Layout(
        padding="2px", height="auto", align_items="center", justify_conten="center", min_width="400px", width="auto"
    )
)

# param_one = widgets.FloatText(value=0, description="Parameter1: ", disabled=False, layout=Layout(margin="5px 10px"))
# param_two = widgets.FloatText(value=0, description="Parameter2: ", disabled=False, layout=Layout(margin="5px 10px"))
outcome = widgets.IntText(value=0, description="Outcome: ", disabled=False, layout=Layout(margin="5px 10px"))
# param_one, param_two, outcome
param_box_tab = widgets.VBox([], layout=Layout(max_width="350px")) 



interactive_btn_labels = ['New Parameters', 'Update Model', 'Send to Client']

interactive_buttons = [
    widgets.Button(description=f"{interactive_btn_labels[i]}",
    disabled=False,
    button_style='',
    tooltip='Click me',) for i in range(3)]

inteact_btn_box = widgets.Box(interactive_buttons)
interactive_buttons[2].disabled = True
interactive_buttons[1].disabled = True

interactive_container = widgets.VBox([inteact_btn_box, param_box_tab, param_output])


def get_next(d):  # ask
    global server
    global trial_params
    global strat
    global db_config_str
    
    interactive_buttons[1].disabled = False
    
    param_output.clear_output()
    ask_message = {"type": "ask", "version": "0.01", "message": ""}
    ask_response = server.versioned_handler(ask_message)["config"]
    param_output.clear_output()
    with param_output:

        children = []
        for name, data in ask_response.items():
            children.append(widgets.FloatText(value=data[0], description=f"{name}", disabled=False, layout=Layout(margin="5px 10px")))
        
        children.append(outcome)
        param_box_tab.children = children
        




def tell_model(d):  # Tell
    param_output.clear_output()
    
    tell_msg = {}
    for param in param_box_tab.children: 
        tell_msg[param.description] = param.value
    
    outcome_val = outcome.value
           
    try:
        message = {
            "type": "tell",
            "version": "0.01",
            "message": {
                "config": tell_msg,
                "outcome": outcome_val,
            },
        }
        server.versioned_handler(message)
        with param_output:
            print("Received msg [tell]")
        for child in param_box_tab.children: 
            child.value = 0
    except Exception as e:
        with param_output:
            print("Unable to update Model...")
            print(e)
            print(traceback.format_exc())
    
   
    # if server.strat.x is not None:
    #     data = {par: server.strat.x[:, i] for i, par in enumerate(server.parnames)}
    #     data["outcome"] = server.strat.y
    #     data = pd.DataFrame(data)
    #     #  param_output.clear_output()
    #     display(
    #         html(
    #             f"""<div style='height: 200px; overflow: auto; width: 100%'>
    #      <p style="margin: 1px;">Server Data</p>
    #     {data.style.render()}
    #     </div>"""
    #         )
    #     )


# This will just be to send a message to the server
def send_to_client(d):
    param_output.clear_output()
    with param_output:
        print(
            """Send to Client ----> To Do 
        #  sent the outcome info to the server so that it can update its model 
        # server.tell(outcome, trial_params)
        """
        )


interactive_buttons[0].on_click(get_next)
interactive_buttons[1].on_click(tell_model)
interactive_buttons[2].on_click(send_to_client)

# trial_params = None

# def new_parameters_clicked(d):    #ask 
#     global server
#     global trial_params
    
#     param_output.clear_output()
#     with param_output:
# #         trial_params = server.ask()
#         print("""new_parameters clicked! ---> in progress 
#         #ask the server what parameter values to try next 
#         # server.ask() 
#         """)
#     load_logs()


# def update_model_clicked(d):   # Tell 
#     param_output.clear_output()
#     with param_output: 
#         print("""Update Model ----> To Do  
#         # tell the server the outcome 
#         # store the outcome in a variable that a user can update via an input
#         """)


# def send_to_client(d):
#     param_output.clear_output()
#     with param_output: 
#         print("""Send to Client ----> To Do 
#         #  sent the outcome info to the server so that it can update its model 
#         # server.tell(outcome, trial_params)
#         """)

# interactive_buttons[0].on_click(new_parameters_clicked)
# interactive_buttons[1].on_click(update_model_clicked)
# interactive_buttons[2].on_click(send_to_client) 
# ------- View 2 --------
# --> plot view 
plot_output = widgets.Output(layout=Layout(
    padding="2px", 
    height="100%", 
    align_items='center',
    justify_conten="center",
    width="100%"))

# --> table view


table_data_output = widgets.Output(layout=Layout(width='600px',height='400px'))
download_link_output = widgets.Output()

######### DOWNLOAD SECTION ###################
from tempfile import NamedTemporaryFile, TemporaryDirectory
from zipfile import ZipFile
# https://stackoverflow.com/questions/72361140/how-can-i-make-a-temporaty-zip-object-downlaodable-for-user-using-jupyter-notebo
                

default_db_file_path = os.getcwd() + '/databases/default.db'
#        print("Does path exisit?", os.getcwd()+'/databases/default.db', os.path.isdir(os.getcwd()+'/databases/default.db'))
        


def create_download_html(htmlWidget, filename, title = "Click here to download: "):
    # Change widget html temperarily to a font-awesome spinner
    htmlWidget.value = "<i class=\"fa fa-spinner fa-spin fa-2x fa-fw\"></i><span class=\"sr-only\">Loading...</span>"
    
    # Process raw data
    data = open(filename, "rb").read()
    b64 = base64.b64encode(data)
    payload = b64.decode()
    
    # Create and assign html to widget
    html = '<a download="{filename}" class="download-link" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    htmlWidget.value = html.format(payload = payload, title = title+filename, filename = filename)


    
htmlWidget = widgets.HTML(value = '')


def download_link():
    global resume_experiment
    global new_expriment
    
    download_link_output.clear_output() 
    with download_link_output:
        if new_experiment:
#             display(html("""
#             <a href="./databases/default.db" download>Download the Exprimant Database</a>
#             """))
            create_download_html(htmlWidget,'./databases/default.db',"Click here to download: ")
            display(htmlWidget)
#             with ZipFile("databases/new_database.zip", 'w') as zipObj:
#                 zipObj.write('./databases/default.db')
                
#                 display(html("""
#                 <a href="./databases/new_database.zip" download>Download zip db</a>
#                 """))    
        elif resume_experiment: 
            
            relative_path = "./" + "/".join(database_path.split("/")[-2:])
            
            create_download_html(htmlWidget,relative_path,"Click here to download: ")
            display(htmlWidget)
#             with ZipFile("databases/resume_database.zip", 'w') as zipObj:
#                 zipObj.write(relative_path)

#                 display(html("""
#                 <a href="./databases/resume_database.zip" download>Download zip db</a>
#                 """)) 
        


######################################


# def display_data():
#     global db_config_str
#     with table_data_output:
#         uuid = server.db.get_master_records()[0].experiment_id
#         strat = server.get_strat_from_replay()
#         replay_data = server.db.get_replay_for(uuid)
#         print('{:<4}{:>20}{:>20}{:>25}'.format("unique_","message_type","master_talbe_id","message_contents"))

#         for record in replay_data:
#             if record.message_type == 'setup':   
#                 db_config_str = record.message_contents['message']['config_str']
#                 resume_config_value()
#                 print('{:<4}{:>20}{:>20}{:>25}'.format(record.unique_id, 
#                       record.message_type, 
#                       record.master_table_id, 
#                       'view config under Exp details',)) 
#             else: 
#                 print('{:<4}{:>20}{:>20}{:>25}'.format(record.unique_id, 
#                       record.message_type, 
#                       record.master_table_id,
#                       str(record.message_contents['message'])))
            
    
        
#         data = server.get_dataframe_from_replay()
#         display("data", data)

def display_data():
    global db_config_str
    with table_data_output:
        uuid = server.db.get_master_records()[0].experiment_id
        replay_data = server.db.get_replay_for(uuid)
        setup_table_row = ""
        table_rows = ""
        
        for record in replay_data:
            if record.message_type == 'setup':
                db_config_str = record.message_contents['message']['config_str']
                setup_table_row += f"""
                  <tr>
                    <th>{record.unique_id}</th>
                    <th>{record.message_type}</th>
                    <th>{record.master_table_id}</th>
                    <th>view config under Exp details tab</th>
                </tr>
                """
            else: 
                table_rows += f"""
                  <tr>
                    <th>{record.unique_id}</th>
                    <th>{record.message_type}</th>
                    <th>{record.master_table_id}</th>
                    <th>{str(record.message_contents['message'])}</th>
                </tr>
                """
        display(html(f"""
          <div style='height: 400px; overflow: auto;'>
            <table style='width: 100%;'>
                <tr>
                    <th>unique_</th>
                    <th>message_type</th>
                    <th>master_talbe_id</th>
                    <th>message_contents</th>
                </tr>
                {setup_table_row}
                {table_rows}
            </table>
          </div>
        """))

        
# ---> use for new session 

def display_plot():
    global strat
    plot_output.clear_output()
    with plot_output:
#     #------- Error Handling -----
        error = False
        with plot_output:
            plot_output.clear_output()
            if len(ylabel.value) > 35: 
                error = True
    #             raise Exception("ylabel must be less than 35 characters ")
                print("ylabel must be less than 35 characters ")
            elif len(xlabel.value) > 35: 
                error = True
                print("xlabel must be less than 35 characters ")
            elif len(yes_label.value) > 35: 
                error = True
                print("yes_label must be less than 35 characters ")
            elif len(no_label.value) > 35: 
                error = True
                print("no_label must be less than 35 characters ")
   
        if error == False: 
            inputs_output.clear_output() 
        try: 
            if server.strat.model is not None:
                plot_strat(
                    server.strat,
                    xlabel=xlabel.value,
                    ylabel=ylabel.value,
                    no_label=no_label.value,
                    yes_label=yes_label.value,
                    cred_level=cred_level.value,
                    target_level=target_level.value,
                )
            elif server.strat.model == None: 
                print("Collect more data")
        except AttributeError as err:
            print("Collect more data, \n")
            print(err)
            print(traceback.format_exc())
        except NotImplementedError as err:
            print(err)
            print("Only plot 3 pramameters at once")
        except AssertionError as err:
            print("Collect more data, \n")
            print(err)
            print(traceback.format_exc())
        except Exception as e:
            print(e) 
            print(traceback.format_exc())
         


out = widgets.Output(layout=Layout(
    padding="2px", 
    height="100%", 
    align_items='center',
    justify_conten="flex-start",
    margin="0",
    width="100%"
))

exp_details = widgets.Button(
    description='Exp Details',
    disabled=False,
    button_style='',
    tooltip='Click me')
plot_view = widgets.Button(
    description="Plot View",
    disable=False,
    button_style="",
    tooltip="Click me")
table_view = widgets.Button(
    description="Table View",
    disable=False,
    button_style="",
    tooltip="Click me")
logs = widgets.Button(
    description="Logs",
    disable=False,
    button_style="",
    tooltip="Click me")
interactive = widgets.Button(
    description="Interactive",
    disable=False,
    button_style="",
    tooltip="Click me")

navigation = widgets.Box([exp_details,plot_view,table_view,logs,interactive],
                         
                         
layout=Layout(  
        display='flex',
        flex_flow="column",
        width='200px',
        margin="0 2.5%",
        min_width="152px"
        ))
output_header_style = Layout(justify_content="center")
nav_display_style = Layout(display="flex",
        flex_flow="column")


container_one = widgets.Box([
    Label("Exp Details", layout=output_header_style),
    resume_config_output
    ],
    layout=nav_display_style )


container_one_new = widgets.Box([config_container,
    config_buttons, config_output,
],layout=nav_display_style)

plot_containter = widgets.HBox([plot_output,plot_interact_box])
plot_containter.add_class("plot-container")

container_two = widgets.Box([
    Label("Plot view", layout=output_header_style),
    plot_containter],
    layout=Layout(width="100%",display="flex",flex_flow="column"))

container_three = widgets.Box([
    Label("Table View", layout=output_header_style),download_link_output,
    table_data_output],
    layout=nav_display_style)

container_four = widgets.Box([
    Label("Logs", layout=output_header_style),
    logger_output],
    layout=nav_display_style)

container_five = widgets.Box([interactive_container], layout=Layout(margin="0",padding="0"))

nav_container = widgets.Box(
    [navigation,out],
    layout=widgets.Layout(
    display="flex",
    flex_flow='row'
    ))
main_container = widgets.Box(
    [connection_inputs,nav_container],
    layout=widgets.Layout(
    display="flex",
    flex_flow="column",
    marging="auto"
    )
)

main_container.add_class("main_container")

# display(main_container)


def nav_btn_click(b):
    global resume_experiment
    global new_experiment
    show_display = b.description
    out.clear_output()
    if show_display == 'Exp Details':
        with out:
            if new_experiment: 
                display(container_one_new)
            if resume_experiment:  
                resume_config_value()
                display(container_one)
    elif show_display == "Plot View":
        display_plot() 
        with out:
            display(container_two)
    elif show_display == "Table View":
        download_link()
        with out: 
            display(container_three)
    elif show_display == "Logs":
        with out:
            load_logs()
            display(container_four)
    elif show_display == "Interactive":
#         ask_for_params()
        with out: 
            display(container_five)

            
exp_details.on_click(nav_btn_click)    
plot_view.on_click(nav_btn_click)
table_view.on_click(nav_btn_click)
logs.on_click(nav_btn_click)
interactive.on_click(nav_btn_click)


#------ View 1 ------            
uploader = widgets.FileUpload(
    description="Connect to Database",accept=".db", multiple=False, layout=Layout(margin="15px 60px"))

def start_server():
    global strat
    global server

    with error_output:
        try:
            server = AEPsychServer()
            make_config()
            config_output.clear_output()
            with out: 
                display(container_one_new)
            # strat = server.get_strat_from_replay()
            # display_data()
            # if display_plot() == True:
            home_view_container.layout.display = "none"
            main_container.layout.display = "inline"
            display(main_container)
        except Exception as e:
            print("Unable to start a new experiment")
            print(e)
            exc_type, exc_obj, exc_tb = sys.exc_info()
            fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
            print(exc_type, fname, exc_tb.tb_lineno)
    
    
#     config = show_config()
#     global strat
#     global server
#     with error_output:
#         try:
#             server = AEPsychServer(database_path=database_path)
#             strat = server.get_strat_from_replay()
#             print(lent(strat))
#             display_data()
#             if display_plot() == True:
#                 home_view_container.layout.display = 'none' 
#                 main_container.layout.display = 'inline'
#             else:
#                 error_output.clear_output()
#                 if strat != None:
#                     try:
#                         plot_strat(
#                             strat,
#                             xlabel=xlabel.value,
#                             ylabel=ylabel.value,
#                             no_label=no_label.value,
#                             yes_label=yes_label.value,
#                             cred_level=cred_level.value,
#                             target_level=target_level.value,
#                         )
#                     except RuntimeError as err: 
#                         error_output.clear_output()
#                         with error_output:
#                             print("Unable to connect to server. \n")
#                             print(err)
#                     except AssertionError as err: 
#                         print(err)

#                 display(main_container)
#         except sqlalchemy.exc.DatabaseError as err:
#                     error_output.clear_output()
#                     with error_output:
#                         print("Unable to connect to server. \n")
#                         print(err)
#         except Exception as e:
#             print("unable to connect to server...")
#             print(e)
#     #             exc_type, exc_obj, exc_tb = sys.exc_info()
#     #             fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
#     #             print(exc_type, fname, exc_tb.tb_lineno)    
    

    
    
# db_experiments = widgets.Dropdown(
#     options=["No experiment_ids in db file"],
#     value=exp_options[-1],
#     description='Select Experiment:',
#     disabled=False,
# )
   
# db_experiments.observe(on_value_change_input, names="value") 
#             if len(exp_ids): 
#                 db_experiments.options = exp_ids
                
#             display(db_experiments)
#           Error with replaying date
#             print(db_experiments.value)
    
#             if db_experimets.value != "No experiment_ids in db file":

    
def resume_server():
#     global strat
    global server
    with error_output:
        try:
            !true > logs/bayes_opt_server.log
            
            server = AEPsychServer(database_path=database_path)
            exp_ids = [rec.experiment_id for rec in server.db.get_master_records()]
            print("Replaying experiment data...")
              
            server.replay(exp_ids[-1], skip_computations=True)
            server.ask()
            
#             TESTING   
#             strat_id = server.db.get_strat_for(master_id=exp_ids[-1])
#             print(strat_id, "STRAT")
#             resume_message = {"type": "resume", "version": "0.01", "message": {"strat_id":f"{exp_ids}"}}
#             resume_response = server.versioned_handler(resume_message)
#             print(resume_response)            
            
            connection_input_box.layout.display = "none"

            display_data() #loads config on inital render
            resume_config_value()
            with out:
                display(container_one)
            home_view_container.layout.display = 'none' 
            main_container.layout.display = 'inline'
            
        except IndexError as err:
            error_output.clear_output()
            print("""
            File does not contain experiment records.
            Please upload another file. 
            """)
            print(err)
            print(traceback.format_exc())
        except Exception as e:
            print("unable to connect to server...")
            print(e)
            print(traceback.format_exc())

            

def upload():
    global server
    global strat
    global database_path
    global db_file_name
    error_output.clear_output() 
    with error_output:     
        error_output.clear_output()
        if uploader.value == {}:
            print("No file uploaded")
        else:
            for file_name, file in uploader.value.items():
                db_file_name = file_name 
                current_path = os.getcwd()
                external_target_path = current_path + "/databases"
                completeName = None 
                
                if os.path.isdir(external_target_path):
                    completeName = os.path.join(external_target_path, file_name)
                else:
                    print("File upload unsuccessful")

                with open(completeName, "wb") as f:
                    f.write(file["content"])
                    if completeName != None:
                        database_path = completeName
                    resume_server()
                    f.close()


def uploader_on_change(change): 
    global resume_experiment
    
    error_output.clear_output()
    with error_output:
        try: 
            upload()
            resume_experiment = True
        except Exception as e:
            print(e)
            print(traceback.format_exc())

uploader.observe(uploader_on_change, names="value")

error_output = widgets.Output(layout=Layout(
    padding="2px", 
    height="100%", 
    align_items='center',
    justify_conten="center",
    width="100%"))
build_experiment = widgets.Button(
    description='Build Experiment',
    disabled=False,
    button_style='info',
    tooltip='Click me',
    layout=Layout(width='auto', height="auto", margin="15px 60px"))

home_view_container = widgets.Box([
    Label("Build a new experiment, or load data from an old one"), 
    widgets.HBox([build_experiment, uploader], 
                layout=widgets.Layout(
                display="flex",
                justify_content="center",
                margin="30px 0",
                align_items="center"
                )),
    error_output,],
    layout=Layout(
    display="flex",
    flex_flow='column',
    width="100%",
    height="100%",
    margin="60px 0",
    justify_content='center',
    align_items="center",))



wrapper_container = widgets.Box([header,main_container,home_view_container],
                               layout=Layout(display="flex",flex_flow="column"))
main_container.layout.display = 'none'
display(wrapper_container)


def build_experiment_clicked(b): 
    global new_experiment 
    new_experiment = True
    error_output.clear_output()
    with error_output:
        start_server()        
 
    
build_experiment.on_click(build_experiment_clicked)

Output()

Box(children=(HTML(value='\n<div class="nav" style=" \nmargin: 0px;\nbackground-color: #F2F2F2;\nfont-weight: …